Will solve:

https://github.com/CONABIO/kube_sipecam_playground/issues/5

# Set up minikube and use docker image for MAD-Mex in AWS

Will follow: 

* For minikube: [minikube_sipecam/setup](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/setup#aws)

* docker image for MAD-Mex: [kube_sipecam/dockerfiles/MAD_Mex/odc_kale](https://github.com/CONABIO/kube_sipecam/tree/master/dockerfiles/MAD_Mex/odc_kale) and [minikube_sipecam/deployments/MAD_Mex](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/)


Will use [minikube_sipecam/deployments/MAD_Mex/hostpath_pv](https://github.com/CONABIO/kube_sipecam/tree/master/minikube_sipecam/deployments/MAD_Mex/hostpath_pv)

# Set up postgresql instance in AWS

Will follow:

https://github.com/CONABIO/antares3-docker/tree/master/postgresql/local_deployment

```
sudo su
cd /shared_volume
dir=/shared_volume/postgresql_volume_docker
mkdir $dir

git clone https://github.com/CONABIO/antares3-docker.git $dir/antares3-docker

mkdir -p $dir/etc/postgresql
mkdir -p $dir/var/log/postgresql
mkdir -p $dir/var/lib/postgresql

docker run -v $dir/etc/postgresql:/etc/postgresql \
-v $dir/var/log/postgresql:/var/log/postgresql \
-v $dir/var/lib/postgresql:/var/lib/postgresql \
-v $dir/antares3-docker/postgresql/local_deployment/conf/:/home/postgres/conf/ \
-w /home/postgres \
-p 2225:22 -p 5432:5432 --name postgresql-local --hostname postgresql-madmex \
-dit madmex/postgresql-madmex-local:v8 /bin/bash
```

## Init files

**Next commands in jupyterlab**

`~/.datacube.conf`

```
[user]
default_environment: datacube
#default_environment: s3aio_env

[datacube]
db_hostname: 172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
db_database: antares_datacube
db_username: postgres
db_password: postgres


execution_engine.use_s3: False

[s3aio_env]
db_hostname: 172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
db_database: antares_datacube
db_username: postgres
db_password: postgres

#index_driver: s3aio_index

execution_engine.use_s3: False
```

`~/.antares`

```
# Django settings
SECRET_KEY=<key>
DEBUG=True
DJANGO_LOG_LEVEL=DEBUG
ALLOWED_HOSTS=
# Database
DATABASE_NAME=antares_datacube
DATABASE_USER=postgres
DATABASE_PASSWORD=postgres
DATABASE_HOST=172.17.0.61 #check this with docker inspect postgresql-local|grep IPA
DATABASE_PORT=5432
# Datacube
SERIALIZED_OBJECTS_DIR=/root/datacube_ingest/serialized_objects/
INGESTION_PATH=/root/datacube_ingest
#DRIVER=s3aio
DRIVER='NetCDF CF'
#INGESTION_BUCKET=datacube-s2-jalisco-test
# Query and download
USGS_USER=<username>
USGS_PASSWORD=<password>
SCIHUB_USER=
SCIHUB_PASSWORD=
# Misc
BIS_LICENSE=<license>
TEMP_DIR=/shared_volume/temp
#SEGMENTATION_BUCKET=<name of bucket>

```

Install antares

`pip3 install --user git+https://github.com/CONABIO/antares3.git@develop`

Init antares and datacube:

```
~/.local/bin/antares init
datacube -v system init
```


**Create spatial indexes:**

```
apt-get install postgresql-client
psql -h 172.17.0.61 -d antares_datacube -U postgres
CREATE INDEX madmex_predictobject_gix ON public.madmex_predictobject USING GIST (the_geom);
CREATE INDEX madmex_trainobject_gix ON public.madmex_trainobject USING GIST (the_geom);
```

**Check:**

`datacube -v system check`